In [ ]:
# Install required packages
# pip install tensorflow pandas scikit-learn

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 1. Load your data
df = pd.read_csv('/content/stock_news_sentiment_dataset (3).csv')  # <-- Change if needed


In [ ]:
# 2. Preprocessing
# Combine headline + description
df['full_text'] = df['Headline'].astype(str) + " " + df['Description'].astype(str)

# Encode sentiment
df['Sentiment'] = df['Sentiment'].map({'Good': 1, 'Bad': 0})

texts = df['full_text'].values
labels = df['Sentiment'].values
stock_prices = df['Stock Price'].values

# 3. Tokenization
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

# 4. Padding
max_length = 100
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')

# 5. Define Risk Level
def assign_risk(sentiment, stock_price):
    if sentiment == 0 and stock_price < 100:
        return 'High Risk'
    elif sentiment == 1 and stock_price < 100:
        return 'Medium Risk'
    elif sentiment == 0 and stock_price >= 100:
        return 'Medium Risk'
    elif sentiment == 1 and stock_price >= 100:
        return 'Low Risk'

df['Risk_Level'] = df.apply(lambda row: assign_risk(row['Sentiment'], row['Stock Price']), axis=1)

# 6. Split Train / Test
X_train, X_test, y_train, y_test = train_test_split(
    padded_sequences, labels, test_size=0.2, random_state=42, stratify=labels
)

# 7. Build LSTM Model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=64, input_length=max_length))
model.add(LSTM(64, return_sequences=False))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 8. Train Model
model.fit(
    X_train, y_train,
    validation_split=0.2,
    batch_size=4,
    epochs=10,
    verbose=1
)

# 9. Predict on test set
y_pred = (model.predict(X_test) > 0.5).astype(int)
print(f"✅ Test Accuracy: {accuracy_score(y_test, y_pred) * 100:.2f}%")

# 10. FINAL Function for Testing New Input

def predict_sentiment_and_risk(headline, description, stock_price):
    text = headline + " " + description
    seq = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(seq, maxlen=max_length, padding='post')

    sentiment_pred = (model.predict(padded) > 0.5).astype(int)[0][0]

    # Map prediction
    sentiment_label = "Good" if sentiment_pred == 1 else "Bad"

    # Assign Risk
    risk = assign_risk(sentiment_pred, stock_price)

    return sentiment_label, risk

# Example Testing
headline = "3M Reports Record Revenue Growth in Healthcare Division"
description = "The company announced its healthcare segment outperformed analyst expectations with a 10% YoY revenue growth."
stock_price = 128.45

sentiment, risk = predict_sentiment_and_risk(headline, description, stock_price)

print(f"\n📢 Predicted Sentiment: {sentiment}")
print(f"⚡ Assigned Risk Level: {risk}")

Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


480/480 ━━━━━━━━━━━━━━━━━━━━ 23s 41ms/step - accuracy: 0.8569 - loss: 0.4359 - val_accuracy: 0.8729 - val_loss: 0.3782
Epoch 2/10
480/480 ━━━━━━━━━━━━━━━━━━━━ 19s 38ms/step - accuracy: 0.8591 - loss: 0.4054 - val_accuracy: 0.8729 - val_loss: 0.4203
Epoch 3/10
480/480 ━━━━━━━━━━━━━━━━━━━━ 21s 39ms/step - accuracy: 0.8428 - loss: 0.4393 - val_accuracy: 0.8729 - val_loss: 0.3684
Epoch 4/10
480/480 ━━━━━━━━━━━━━━━━━━━━ 22s 41ms/step - accuracy: 0.8397 - loss: 0.3959 - val_accuracy: 0.8687 - val_loss: 0.3818
Epoch 5/10
480/480 ━━━━━━━━━━━━━━━━━━━━ 19s 38ms/step - accuracy: 0.8480 - loss: 0.3693 - val_accuracy: 0.8667 - val_loss: 0.3693
Epoch 6/10
480/480 ━━━━━━━━━━━━━━━━━━━━ 20s 42ms/step - accuracy: 0.8604 - loss: 0.3905 - val_accuracy: 0.8750 - val_loss: 0.3415
Epoch 7/10
480/480 ━━━━━━━━━━━━━━━━━━━━ 18s 38ms/step - accuracy: 0.8885 - loss: 0.3274 - val_accuracy: 0.8938 - val_loss: 0.3102
Epoch 8/10
480/480 ━━━━━━━━━━━━━━━━━━━━ 19s 40ms/step - accuracy: 0.9096 - loss: 0.3050 - val_accurac

In [ ]:
# 1. Predict on Test Set
y_pred_probs = model.predict(X_test)
y_pred = (y_pred_probs > 0.5).astype(int).flatten()

# 2. Build test DataFrame (based on original df)
# First, recreate original padded sequences' indexes
test_indexes = y_test.index if hasattr(y_test, 'index') else np.arange(len(y_test))

# Select corresponding rows from original DataFrame
df_test = df.iloc[test_indexes].copy()  # make a copy to not mess up original

# 3. Add Predictions
df_test['True_Sentiment_Label'] = df_test['Sentiment'].map({1: 'Good', 0: 'Bad'})
df_test['Predicted_Sentiment'] = y_pred
df_test['Predicted_Sentiment_Label'] = df_test['Predicted_Sentiment'].map({1: 'Good', 0: 'Bad'})

# 4. Assign Risk based on Predicted Sentiment
df_test['Predicted_Risk_Level'] = df_test.apply(lambda row: assign_risk(row['Predicted_Sentiment'], row['Stock Price']), axis=1)

# 5. Save full table
df_test.to_csv('full_test_results_with_predictions.csv', index=False)

print("✅ Full test results saved to 'full_test_results_with_predictions.csv' successfully!")


19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
✅ Full test results saved to 'full_test_results_with_predictions.csv' successfully!


In [ ]:
!pip install yfinance beautifulsoup4 requests lxml newspaper3k


In [ ]:
!pip install lxml_html_clean


In [ ]:
!pip install newspaper3k --no-cache-dir --force-reinstall


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 19.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 97.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 143.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 188.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 186.0/186.0 kB 145.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 162.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 69.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 81.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:

import yfinance as yf
import requests
from bs4 import BeautifulSoup
from newspaper import Article

# Function to fetch stock price
def fetch_stock_price(ticker):
    try:
        stock = yf.Ticker(ticker)
        todays_data = stock.history(period='1d')
        return todays_data['Close'].iloc[0]
    except Exception as e:
        print(f"❗ Error fetching stock price: {e}")
        return None

# Function to fetch latest headline + link
def fetch_latest_headline_and_link(ticker):
    try:
        url = f"https://finviz.com/quote.ashx?t={ticker}"
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, "html.parser")
        news_table = soup.find(id="news-table")
        first_news = news_table.find('tr')

        if first_news and first_news.a:
            headline = first_news.a.text
            link = first_news.a['href']
            return headline, link
        else:
            return "No news found", None
    except Exception as e:
        print(f"❗ Error fetching news: {e}")
        return "No news found", None

# Function to extract article description from link
def fetch_article_description(link):
    if not link:
        return "No description available."
    try:
        article = Article(link)
        article.download()
        article.parse()
        return article.text
    except Exception as e:
        print(f"❗ Error fetching article description: {e}")
        return "No description available."

# Main function
def predict_from_stock_input_full():
    print("📋 Please enter your stock symbol:")

    # Take stock symbol input
    company = input("Enter Stock Symbol (e.g., MMM): ").strip().upper()

    # Fetch stock price
    stock_price = fetch_stock_price(company)
    if stock_price is None:
        print("❗ Could not fetch stock price. Exiting.")
        return

    # Fetch latest headline + link
    headline, link = fetch_latest_headline_and_link(company)

    if link:
        description = fetch_article_description(link)
    else:
        description = headline  # fallback if no link

    print(f"\n🔎 Fetched Data for {company}:")
    print(f"Stock Price: {stock_price}")
    print(f"Latest News Headline: {headline}")
    print(f"News Description: {description[:200]}...")  # Print first 200 chars

    # Combine headline + description
    full_text = headline + " " + description

    # Tokenize and pad
    seq = tokenizer.texts_to_sequences([full_text])
    padded = pad_sequences(seq, maxlen=max_length, padding='post')

    # Predict sentiment
    sentiment_pred = (model.predict(padded) > 0.5).astype(int)[0][0]
    sentiment_label = "Good" if sentiment_pred == 1 else "Bad"

    # Predict risk
    risk_level = assign_risk(sentiment_pred, stock_price)

    print("\n✅ Prediction Complete:")
    print(f"Company: {company}")
    print(f"Predicted Sentiment: {sentiment_label}")
    print(f"Assigned Risk Level: {risk_level}")

# Call the final full function
predict_from_stock_input_full()


📋 Please enter your stock symbol:
Enter Stock Symbol (e.g., MMM): AAPL
❗ Error fetching article description: Article `download()` failed with Invalid URL '/news/18545/think-you-know-apple-heres-1-lesser-known-fact-you-shouldnt-overlook': No scheme supplied. Perhaps you meant https:///news/18545/think-you-know-apple-heres-1-lesser-known-fact-you-shouldnt-overlook? on URL /news/18545/think-you-know-apple-heres-1-lesser-known-fact-you-shouldnt-overlook

🔎 Fetched Data for AAPL:
Stock Price: 188.3800048828125
Latest News Headline: Think You Know Apple? Here's 1 Lesser-Known Fact You Shouldn't Overlook.
News Description: No description available....
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step

✅ Prediction Complete:
Company: AAPL
Predicted Sentiment: Good
Assigned Risk Level: Low Risk


Recommendation

In [ ]:

# STEP 1: Predefined List of Popular Stocks to scan
popular_stocks = ['AAPL', 'MSFT', 'NVDA', 'TSLA', 'AMZN', 'META', 'GOOGL', 'NFLX', 'AMD', 'CRM']

# STEP 2: Recommendation Function
def recommend_top_stocks():
    results = []

    print("🔎 Scanning popular stocks for recommendations...")

    for ticker in popular_stocks:
        try:
            stock_price = fetch_stock_price(ticker)
            if stock_price is None:
                continue

            headline, link = fetch_latest_headline_and_link(ticker)

            if link:
                description = fetch_article_description(link)
            else:
                description = headline  # fallback

            full_text = headline + " " + description

            seq = tokenizer.texts_to_sequences([full_text])
            padded = pad_sequences(seq, maxlen=max_length, padding='post')

            sentiment_pred = (model.predict(padded) > 0.5).astype(int)[0][0]
            sentiment_score = model.predict(padded)[0][0]  # Get probability between 0 and 1
            sentiment_label = "Good" if sentiment_pred == 1 else "Bad"

            risk_level = assign_risk(sentiment_pred, stock_price)

            results.append({
                'Ticker': ticker,
                'Stock Price': stock_price,
                'Predicted Sentiment': sentiment_label,
                'Sentiment Probability': sentiment_score,
                'Risk Level': risk_level
            })

            print(f"✅ {ticker}: Sentiment={sentiment_label}, Risk={risk_level}, Price=${stock_price:.2f}")

        except Exception as e:
            print(f"❗ Error processing {ticker}: {e}")
            continue

    # STEP 3: Create DataFrame
    df_results = pd.DataFrame(results)

    if df_results.empty:
        print("❗ No stocks could be recommended.")
        return

    # STEP 4: Filter Low/Medium Risk and Good Sentiment
    df_filtered = df_results[
        (df_results['Risk Level'].isin(['Low Risk', 'Medium Risk'])) &
        (df_results['Predicted Sentiment'] == 'Good')
    ]

    # STEP 5: Sort by sentiment probability (confidence) and pick Top 3
    top_picks = df_filtered.sort_values(by='Sentiment Probability', ascending=False).head(3)

    if top_picks.empty:
        print("❗ No Low/Medium Risk Good stocks found.")
        return

    print("\n🏆 Top 3 Recommended Stocks to Invest In:")
    print(top_picks[['Ticker', 'Stock Price', 'Predicted Sentiment', 'Risk Level']])

# STEP 6: Call the recommendation function
recommend_top_stocks()


🔎 Scanning popular stocks for recommendations...
❗ Error fetching article description: Article `download()` failed with Invalid URL '/news/18545/think-you-know-apple-heres-1-lesser-known-fact-you-shouldnt-overlook': No scheme supplied. Perhaps you meant https:///news/18545/think-you-know-apple-heres-1-lesser-known-fact-you-shouldnt-overlook? on URL /news/18545/think-you-know-apple-heres-1-lesser-known-fact-you-shouldnt-overlook
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
✅ AAPL: Sentiment=Good, Risk=Low Risk, Price=$188.38
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
✅ MSFT: Sentiment=Good, Risk=Low Risk, Price=$359.84
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
✅ NVDA: Sentiment=Good, Risk=Medium Risk, Price=$94.31
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
✅ TSLA: Sentiment=Good, Risk=Low Risk, Price=$239.43
❗ Error fetching article description: Article `dow